In [47]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

# import working with data libraries
import pandas as pd
import numpy as np

# import seaborn
import seaborn as sns

# import matplotlib
import matplotlib.pyplot as plt # Typical way of import MPL
import matplotlib as mpl # This line is used less frequently

In [39]:
### load the review data

review_data = []
with gzip.open('Software.json.gz') as f:
    for l in f:
        review_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of reviews
print(len(review_data))

# first row of the list
print(review_data[10])

459436
{'overall': 2.0, 'vote': '3', 'verified': True, 'reviewTime': '05 3, 2009', 'reviewerID': 'A1CUJ6ZT82H7LS', 'asin': '0132147556', 'style': {'Format:': ' Paperback'}, 'reviewerName': 'Brian', 'reviewText': "Maybe it's just me (I have no marketing background but desperately want to learn for my start-up) but I cannot get hardly anything out of this text. I have tried very hard to tread through the writing and learn something useful but chapter after chapter seems to be the exact same thing... overly wordy, rambling & unnecessarily academic writing with no comprehensible message behind it, no strategies I can apply to my own business and no organization that I can make sense of. Feels almost like I'm reading a paper a college student BSed their way through with a whole bunch of long words and cryptic sentences in order to sound impressive. What a rip-off. I paid $70 for this and it has been the most useless book I've paid for in my research process (and I've read 16 other books thu

In [59]:
# convert list into pandas dataframe

all_products = {set}
df_review = pd.DataFrame.from_dict(review_data)

#filter out the non-verified reviews, and leave only the columns we want
df_review = df_review[df_review['verified']==True][["overall", "reviewTime", "asin", "reviewText", "summary"]]
df_review = df_review.rename(columns={"asin": "product_id", "overall": "rating"})
print(len(df_review))
df_review.head()
for index, rating, reviewTime, product_id, reviewText, summary in df_review.itertuples():
    all_products.add(product_id)
print(all_products)

309345
{'B001DYJ5D0', 'B00A9MM55M', 'B009JWEQUK', 'B0007Z70WE', 'B00DRPRTDG', 'B00009VRFK', 'B00EINVIFQ', 'B00DHYJ5MK', 'B00GWSJD1G', 'B000KZF3XE', 'B002NX0N0O', 'B000FETUJ8', 'B00002S7RJ', 'B005NQ745K', 'B008GZZDKI', 'B000HU0ONQ', 'B000AMPJPY', 'B00AOHDC5E', 'B00LLQMC3K', 'B0028PV4JQ', 'B001GJUEOG', 'B00NUEMUL0', 'B01C3IMVNK', 'B001AUEIU2', 'B002EAMAOS', 'B01EB06RD8', 'B0010KEFSW', 'B00QQU9I56', 'B00005MJTY', 'B001O3PGY2', 'B006W1GT2Y', 'B0009EUA2K', 'B00VAAMUXO', 'B017OPDPAU', 'B0002BQMQI', 'B00OF2A30Q', 'B00U3Z5SYA', 'B000BMBLAA', 'B0000A58IY', 'B003URW7OM', 'B000CH1KJ6', 'B000HCVR44', 'B00198JQW0', 'B003N1IOUG', 'B00003IEKB', '1597461156', 'B000M15HFA', 'B0000TN0YY', 'B00C79IYO6', 'B0007ZF3GO', 'B000069K0C', 'B0068TJE4A', 'B00P6VWK1U', 'B013HA2B9W', 'B00ELTY3QI', 'B000068WXU', 'B014M2K27W', 'B00W2VEMW2', 'B000WR2F3G', 'B01GGUTDTG', 'B00005N9AF', 'B0002DOE7K', '0981958559', 'B00006CRUM', 'B003DWRA2I', '3959828233', 'B000056B2D', 'B00094OT7C', 'B009ZOMD0C', 'B00ULP2UDY', 'B0019MUQFC'

In [45]:
### load the mega data

mega_data = []
with gzip.open('meta_Software.json.gz') as f:
    for l in f:
        mega_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(mega_data))

# first row of the list
print(mega_data[1])

26790
{'category': [], 'tech1': '', 'description': ['', '<b>Latin rhythms that will get your kids singing in Spanish</b>', '<i>Sing, Watch, and Learn Spanish</i> helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing along with simple melodies. This charming DVD contains 16 music videos featuring kids engaged in fun activities, from visiting animals at the zoo to comparing clothing sizes in grandmas closet.', 'Each video features an original song of authentic Latin rhythms that gets kids singing along with the children on screen. As they watch, listen, and sing along, kids absorb 300 Spanish words, each of which is shouted out in a song and displayed as a subtitle on screen.', '', '', ''], 'fit': '', 'title': 'Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish', 'also_buy': [], 'tech2': 

In [61]:
# convert list into pandas dataframe

df_product = pd.DataFrame.from_dict(mega_data)

#filter out the non-verified reviews, and leave only the columns we want
df_product = df_product[["asin", "rank"]]
df_product = df_product.rename(columns={"asin": "product_id"})
print(len(df_product))
df_product.head()
# TODO:filter out the products that has reviews, rank in software
# if(product_id)

26790


,product_id,rank
0,0030672120,"25,550 in Software ("
1,0071480935,"15,792 in Software ("
2,007329506X,"16,900 in Software ("
3,0073513458,"12,986 in Software ("
4,0073525758,"14,861 in Software ("


In [43]:
# ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)

# df3 = df.fillna('')
# df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
# df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
# print(len(df4))
# print(len(df5))